In [ ]:
# import lib
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import cifar10

# define block resnet
def block_resnet(input, filter, stride):
  x_i = input

  x = Conv2D(filter, kernel_size=3, strides=stride, padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation(tf.keras.activations.relu)(x)

  x = Conv2D(filter, kernel_size=3, strides=1, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation(tf.keras.activations.relu)(x)

  x = Conv2D(filter, kernel_size=1, strides=stride)(x)
  x = BatchNormalization()(x)

  x_i = Conv2D(filter, kernel_size=1, strides=stride)(x_i)
  x_i = BatchNormalization()(x_i)

  # add for skip layers
  x = tf.keras.layers.Add()([x_i, x])
  x = Activation(tf.keras.activations.relu)(x)

  return x

# define main def for resnet
def resnet18(input_shape, classes):
  # first layer
  input_layer = tf.keras.Input(shape=input_shape)

  # second layer
  x = Conv2D(64, kernel_size=3, padding='same')(input_layer)
  x = BatchNormalization()(x)
  x = Activation(tf.keras.activations.relu)(x)

  x = MaxPooling2D(pool_size=2, strides=2)(x)

  # call the block
  x = block_resnet(x, 64, 1)
  x = block_resnet(x, 64, 1)
  x = block_resnet(x, 128, 1)
  x = block_resnet(x, 128, 1)
  x = block_resnet(x, 256, 1)
  x = block_resnet(x, 256, 1)
  x = block_resnet(x, 512, 1)
  x = block_resnet(x, 512, 1)

  x = GlobalAveragePooling2D()(x)
  x = Flatten()(x)

  output = Dense(classes, activation='softmax')(x)

  model = tf.keras.Model(inputs=input_layer, outputs=output, name='resnet_model')

  return model

# get dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# make an object
input_shape = (32, 32, 3)
num_classes = 10
r18 = resnet18(input_shape, num_classes)

r18.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# for increase speed of train
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(50000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# fit on data
history = r18.fit(train_dataset, epochs=10)

# test
r18.evaluate(test_dataset)

Epoch 1/10
1563/1563 [==============================] - 170s 96ms/step - loss: 1.4107 - accuracy: 0.4824
Epoch 2/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.9915 - accuracy: 0.6463
Epoch 3/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.7926 - accuracy: 0.7259
Epoch 4/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.6657 - accuracy: 0.7704
Epoch 5/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.5775 - accuracy: 0.7996
Epoch 6/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.5113 - accuracy: 0.8240
Epoch 7/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.4531 - accuracy: 0.8439
Epoch 8/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.4076 - accuracy: 0.8594
Epoch 9/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.3616 - accuracy: 0.8757
Epoch 10/10
313/313 [==============================] - 

[0.6560556888580322, 0.7912999987602234]